# Print Categories, Sub-Categories along with Metadata

In [5]:
import difflib as dl
import xlrd
import xlwt
import pandas as pd
import time
import os
from nltk import ngrams
import nltk
import warnings
import re 
import logging
from openpyxl import Workbook, load_workbook
warnings.filterwarnings("ignore", category=DeprecationWarning) 

start = time.time()
keyDir = r'C:\Users\T01144\Desktop\Test Cases\Tag Explorer\Keywords\Keywords2.xlsx'
#inpDir = r'C:\Users\T01144\Desktop\Test Cases\Tag Explorer\Input - Text\a'
inpDir = r'\\brdieshnas1\AMS\SOFTWARE_REPOSITORY\Python\Auchinto Chatterjee\Autocorrect + Categorise\Well_Refined_TXT'
workbook = xlrd.open_workbook(keyDir)
dfKey = pd.read_excel(keyDir, sheetname='Keywords')
printPath = r'C:\Users\T01144\Desktop\Test Cases\Tag Explorer\Output\PythonExport.xls'
wb = xlwt.Workbook()
ws = wb.add_sheet('Sheet1')
global row
row = 0
ws.write(row,0,"File")
ws.write(row,1,"Category")
ws.write(row,2,"Sub-Category")
ws.write(row,3,"Keyword")
ws.write(row,4,"Value")

def PrintException():
    import sys
    import linecache
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))


class ExtractData():
    
    def __init__(self,Key_list,Input_Path):
        self.Key_list = Key_list
        self.Input_Path = Input_Path
        
    def read_text(self):
        
        text = self.Input_Path.read()
        a = None
        for line in text.splitlines():
            a = self.match_keyword(line,text)
            if a!= None:
                return a
       
    def match_keyword(self,line,text):
        
        starting_index = []
        for key in self.Key_list:
            if key in line:
                match_iter = re.finditer(key, line)

                for match in match_iter:
                    matched_text = [match.group(0)]
                    x = match.start()
                   
                    starting_index.append(x)       
            else:
                starting_index.append(-1)
        sort_string = sorted(starting_index)
        

        length = len(sort_string)
      
        for index,obj in enumerate(sort_string):
            if obj != -1:
                c = starting_index.index(obj)
                   
                if (index+1) < length:
                    next = sort_string[index + 1]
                else:
                    next = len(text)
              
                first = len(self.Key_list[c]) + obj
                
                print(self.Key_list[c],":",line[first:next])
                return (line[first:next])
                
def similar(a, b):
    return dl.SequenceMatcher(None, a, b).ratio()

def readfile() :
    try:
        for root, dirs, filenames in os.walk(inpDir):
            for f in filenames :
                print("-"*40)
                print('\033[1m' + "For", f + '\033[0m')
                file1 = (os.path.join(inpDir, f)) 
                Input_Path = open(file1,'r')
                readtext(Input_Path,file1,f)
                #print("")
    except Exception as e:
        print("Problem in readfile():", e)

#Reading a file line by line            
def readtext(Input_Path,file1,f):
    try:
        mean = 0
        count = 0
        with open(file1) as ff: 
            lines = ff.readlines()   

            for i in range(0, len(lines)):
               
                
                line = lines[i]
                
                if line.strip() == '':
                    continue
                #line1 = nltk.word_tokenize(line)
                #print('\033[1m' + "Line:",line+ '\033[0m')
                max1=0
                found = 0
                for iterateKeys in range(0, len(dfKey['Keywords'])):
                    n = len(dfKey['Keywords'][iterateKeys].strip().split(" "))
                    
                    sixgrams = ngrams(line.split(), n)
                    word = None
                    
                    for grams in sixgrams:
                        
                        str1 = ' '.join(grams)
                        key=str1
                        keyword1 = dfKey['Keywords'][iterateKeys]
                                         
                        max1 = similar(key.lower(), keyword1.lower())
                        
                        
                        if max1>0.8:
                            count+=1
                            mean+=max1
                            #print('\033[1m' + "Line:",line+ '\033[0m')
                            print("Keyword:",dfKey['Keywords'][iterateKeys])
                            #print("Length:", n)
                            #print("Grams:",grams)
                            word = keyword1 
                            #print(key, " ",max1," ", word)
                            #print(dl.get_close_matches(key,dfKey['Keywords']))
                             
                            found=1  
                            
                            try:
                                for iterateKeys in range(0, len(dfKey['Keywords'])):
                                    list3 = []
                                    #print(type(dfKey['Keywords'][iterateKeys]))
                                    if dfKey['Keywords'][iterateKeys].lower() in word.lower() :
                                        #print(iterateKeys)             
                                        list3.append(dfKey['Keywords'][iterateKeys])
                                        print("Sub Category:", dfKey['Sub-Category'][iterateKeys])
                                        print("Category:",dfKey['Category'][iterateKeys])
                                        obj = ExtractData(list3,Input_Path)
                                        val = obj.read_text()
                                        print(val)
                                        
                                        global row
                                        row+=1
                                        ws.write(row,0,f)
                                        ws.write(row,1,dfKey['Category'][iterateKeys])
                                        ws.write(row,2,dfKey['Sub-Category'][iterateKeys])
                                        ws.write(row,3,dfKey['Keywords'][iterateKeys])
                                        ws.write(row,4,val)
                                        print("")
                                        
                            except Exception as e:
                                PrintException()
                
    except Exception as e:
        print("Exception:",e)
    
    print("")
    if count==0:
        print("No Keyword")
    else:
        mean=mean/count
    #print("Count:", count)
    #print("Mean:",mean)
    
    #Input_Path1 = Input_Path
    
    
    wb.save(printPath)
if __name__ == '__main__':
    readfile()
    
    
    end = time.time()
    print("\nExecution Time:",end-start)

----------------------------------------
For Baker Hughes_2001.txt
Keyword: Date
Sub Category: LWD/MWD
Category: Well Logs
None

Keyword: Core
Sub Category: Annotated
Category: Well Logs
None

Keyword: Casing Record
Sub Category: Cased Hole
Category: Well Logs
None


----------------------------------------
For Baker Hughes_2012.txt
Keyword: Date
Sub Category: LWD/MWD
Category: Well Logs
None

Keyword: LWD
Sub Category: LWD/MWD
Category: Well Logs
None

Keyword: LWD
Sub Category: LWD/MWD
Category: Well Logs
None

Keyword: Bit Size
Sub Category: MUD/SAMPLE LOG
Category: Well Logs
None

Keyword: MUD
Sub Category: MUD/SAMPLE LOG
Category: Well Logs
None

Keyword: Date
Sub Category: LWD/MWD
Category: Well Logs
None

Keyword: MUD
Sub Category: MUD/SAMPLE LOG
Category: Well Logs
None

Keyword: SAMPLE
Sub Category: MUD/SAMPLE LOG
Category: Well Logs
None


----------------------------------------
For Haliburton_1946.txt
Keyword: SWC
Sub Category: Annotated
Category: Well Logs
SWC :   
  

Key

# Print Categories, Sub-Categories only

In [33]:
import difflib as dl
import xlrd
import xlwt
import pandas as pd
import time
import os
from nltk import ngrams
import nltk
import warnings
import re 
import logging
from openpyxl import Workbook, load_workbook
warnings.filterwarnings("ignore", category=DeprecationWarning) 

start = time.time()
keyDir = r'C:\Users\T01144\Desktop\Test Cases\Tag Explorer\Keywords\Keywords2.xlsx'
inpDir = r'C:\Users\T01144\Desktop\Test Cases\Tag Explorer\Input - Text\a'
workbook = xlrd.open_workbook(keyDir)
dfKey = pd.read_excel(keyDir, sheetname='Keywords')
printPath = r'C:\Users\T01144\Desktop\PythonExport2.xls'
wb = xlwt.Workbook()
ws = wb.add_sheet('Sheet1')
global row
row = 0
ws.write(row,0,"File")
ws.write(row,1,"Category")
ws.write(row,2,"Sub-Category")
ws.write(row,3,"Keyword")

def PrintException():
    import sys
    import linecache
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))


class ExtractData():
    
    def __init__(self,Key_list,Input_Path):
        self.Key_list = Key_list
        self.Input_Path = Input_Path
        
    def read_text(self):
        
        text = self.Input_Path.read()

        for line in text.splitlines():
            a = self.match_keyword(line,text)
            
    def match_keyword(self,line,text):
        
        starting_index = []
        for key in self.Key_list:
            if key in line:
                match_iter = re.finditer(key, line)

                for match in match_iter:
                    matched_text = [match.group(0)]
                    x = match.start()
                   
                    starting_index.append(x)       
            else:
                starting_index.append(-1)
        sort_string = sorted(starting_index)
        

        length = len(sort_string)
      
        for index,obj in enumerate(sort_string):
            if obj != -1:
                c = starting_index.index(obj)
                   
                if (index+1) < length:
                    next = sort_string[index + 1]
                else:
                    next = len(text)
              
                first = len(self.Key_list[c]) + obj
                
                print(self.Key_list[c],":",line[first:next])
                                
def similar(a, b):
    return dl.SequenceMatcher(None, a, b).ratio()

def readfile() :
    try:
        for root, dirs, filenames in os.walk(inpDir):
            for f in filenames :
                print("-"*40)
                print('\033[1m' + "For", f + '\033[0m')
                file1 = (os.path.join(inpDir, f)) 
                Input_Path = open(file1,'r')
                readtext(Input_Path,file1,f)
                #print("")
    except Exception as e:
        print("Problem in readfile():", e)

#Reading a file line by line            
def readtext(Input_Path,file1,f):
    try:
        mean = 0
        count = 0
        with open(file1) as ff: 
            lines = ff.readlines()   

            for i in range(0, len(lines)):
               
                
                line = lines[i]
                
                if line.strip() == '':
                    continue
                #line1 = nltk.word_tokenize(line)
                #print('\033[1m' + "Line:",line+ '\033[0m')
                max1=0
                found = 0
                for iterateKeys in range(0, len(dfKey['Keywords'])):
                    n = len(dfKey['Keywords'][iterateKeys].strip().split(" "))
                    
                    sixgrams = ngrams(line.split(), n)
                    word = None
                    
                    for grams in sixgrams:
                        
                        str1 = ' '.join(grams)
                        key=str1
                        keyword1 = dfKey['Keywords'][iterateKeys]
                                         
                        max1 = similar(key.lower(), keyword1.lower())
                        
                        
                        if max1>0.8:
                            count+=1
                            mean+=max1
                            #print('\033[1m' + "Line:",line+ '\033[0m')
                            print("Keyword:",dfKey['Keywords'][iterateKeys])
                            #print("Length:", n)
                            #print("Grams:",grams)
                            word = keyword1 
                            #print(key, " ",max1," ", word)
                            #print(dl.get_close_matches(key,dfKey['Keywords']))
                             
                            found=1  
                            
                            try:
                                for iterateKeys in range(0, len(dfKey['Keywords'])):
                                    list3 = []
                                    #print(type(dfKey['Keywords'][iterateKeys]))
                                    if dfKey['Keywords'][iterateKeys].lower() in word.lower() :
                                        #print(iterateKeys)             
                                        list3.append(dfKey['Keywords'][iterateKeys])
                                        print("Sub Category:", dfKey['Sub-Category'][iterateKeys])
                                        print("Category:",dfKey['Category'][iterateKeys])
                                        obj = ExtractData(list3,Input_Path)
                                        obj.read_text()
                                        
                                        global row
                                        row+=1
                                        ws.write(row,0,f)
                                        ws.write(row,1,dfKey['Category'][iterateKeys])
                                        ws.write(row,2,dfKey['Sub-Category'][iterateKeys])
                                        ws.write(row,3,dfKey['Keywords'][iterateKeys])

                                        print("")
                                        
                            except Exception as e:
                                PrintException()
                
    except Exception as e:
        print("Exception:",e)
    
    print("")
    if count==0:
        print("No Keyword")
    else:
        mean=mean/count
    #print("Count:", count)
    #print("Mean:",mean)
    
    #Input_Path1 = Input_Path
    
    
    wb.save(printPath)
if __name__ == '__main__':
    readfile()
    
    
    end = time.time()
    print("\nExecution Time:",end-start)

----------------------------------------
For Baker Hughes_2001.txt
Keyword: Date
Sub Category: LWD/MWD
Category: Well Logs

Keyword: Casing Record
Sub Category: Cased Hole
Category: Well Logs


----------------------------------------
For Baker Hughes_2012.txt
Keyword: Date
Sub Category: LWD/MWD
Category: Well Logs

Keyword: Bit Size
Sub Category: MUD/SAMPLE LOG
Category: Well Logs

Keyword: MUD
Sub Category: MUD/SAMPLE LOG
Category: Well Logs

Keyword: MUD
Sub Category: MUD/SAMPLE LOG
Category: Well Logs

Keyword: SAMPLE
Sub Category: MUD/SAMPLE LOG
Category: Well Logs


----------------------------------------
For Haliburton_1946.txt

No Keyword
----------------------------------------
For Haliburton_1993.txt
Keyword: Bit Size
Sub Category: MUD/SAMPLE LOG
Category: Well Logs

Keyword: SAMPLE
Sub Category: MUD/SAMPLE LOG
Category: Well Logs


----------------------------------------
For Haliburton_2008.txt
Keyword: SAMPLE
Sub Category: MUD/SAMPLE LOG
Category: Well Logs


------------

In [18]:
import re 
import logging 

class ExtractData():
    
    def __init__(self,Key_list,Input_Path):
        self.Key_list = Key_list
        self.Input_Path = Input_Path
        
    def read_text(self):
        
        text = self.Input_Path.read()
       
        for line in text.splitlines():
            self.match_keyword(line,text)
       
    def match_keyword(self,line,text):
        
        starting_index = []
        for key in self.Key_list:
            if key in line:
                match_iter = re.finditer(key, line)

                for match in match_iter:
                    matched_text = [match.group(0)]
                    x = match.start()
                    print(matched_text)
                   
                    
            else:
                starting_index.append(-1)
        sort_string = sorted(starting_index)
        
        

        length = len(sort_string)
      
        for index,obj in enumerate(sort_string):
            print(index,obj)
            if obj != -1:
                c = starting_index.index(obj)
                
                if (index+1) < length:
                    next = sort_string[index + 1]
                else:
                    next = len(text)
              
                first = len(self.Key_list[c]) + obj
                
                print(self.Key_list[c],line[first:next])
                
Key_list = ['DATE']

Input_Path = open(r'\\brdieshnas1\AMS\SOFTWARE_REPOSITORY\Python\Auchinto Chatterjee\Autocorrect + Categorise\Well_Refined_TXT\Baker Hughes_2001.txt','r')

obj = ExtractData(Key_list,Input_Path)
obj.read_text()

0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
0 -1
